In [2]:
import requests
from bs4 import BeautifulSoup

In [9]:
#creating the object to hold to page source content
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
#print(page)

#creating an object to store the page source in proper html format
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup)

#storing page source belonging to a prticular id tag on the page
seven_day = soup.find(id="seven-day-forecast")
#print(seven_day)

#finding all instances of a particular class tag within the id and storing it to object
forecast_items = seven_day.find_all(class_="tombstone-container")
#print(forecast_items)

tonight = forecast_items[0]
print(tonight.prettify())


<div class="tombstone-container">
 <p class="period-name">
  Overnight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Overnight: Mostly clear, with a low around 40. Southeast wind 3 to 6 mph. " class="forecast-icon" src="newimages/medium/nfew.png" title="Overnight: Mostly clear, with a low around 40. Southeast wind 3 to 6 mph. "/>
 </p>
 <p class="short-desc">
  Mostly Clear
 </p>
 <p class="temp temp-low">
  Low: 40 °F
 </p>
</div>


In [10]:
#extracting the values of sub tags
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp temp-low").get_text()

print(period)
print(short_desc)
print(temp)

Overnight
Mostly Clear
Low: 40 °F


In [11]:
#extracting the title part of the img tag
img = tonight.find("img")
desc = img['title']

print(desc)

Overnight: Mostly clear, with a low around 40. Southeast wind 3 to 6 mph. 


In [12]:
#extracting all values for the period tag from the entire object
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
print(periods)

['Overnight', 'Thursday', 'ThursdayNight', 'Friday', 'FridayNight', 'Saturday', 'SaturdayNight', 'Sunday', 'SundayNight']


In [13]:
#similarly extracting the other tags also from the entire object
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print()
print(temps)
print()
print(descs)


['Mostly Clear', 'Sunny', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Mostly Sunny', 'Slight ChanceRain', 'Chance Rain', 'Slight ChanceRain']

['Low: 40 °F', 'High: 56 °F', 'Low: 40 °F', 'High: 60 °F', 'Low: 42 °F', 'High: 64 °F', 'Low: 47 °F', 'High: 63 °F', 'Low: 48 °F']

['Overnight: Mostly clear, with a low around 40. Southeast wind 3 to 6 mph. ', 'Thursday: Sunny, with a high near 56. Light and variable wind becoming west 9 to 14 mph in the afternoon. ', 'Thursday Night: Mostly clear, with a low around 40. West wind 6 to 11 mph becoming light north northwest  after midnight. ', 'Friday: Sunny, with a high near 60. Calm wind becoming west northwest 5 to 7 mph in the afternoon. ', 'Friday Night: Mostly clear, with a low around 42. West northwest wind 5 to 8 mph becoming calm  in the evening. ', 'Saturday: Mostly sunny, with a high near 64.', 'Saturday Night: A 20 percent chance of rain after 10pm.  Mostly cloudy, with a low around 47.', 'Sunday: A chance of rain.  Partly sunny, with a

In [14]:
#combining all tags and creating a dataframe out of it
import pandas as pd
weather = pd.DataFrame({
        "period": periods,
         "short_desc": short_descs,
         "temp": temps,
         "desc":descs
    })
weather

,period,short_desc,temp,desc
0,Overnight,Mostly Clear,Low: 40 °F,"Overnight: Mostly clear, with a low around 40...."
1,Thursday,Sunny,High: 56 °F,"Thursday: Sunny, with a high near 56. Light an..."
2,ThursdayNight,Mostly Clear,Low: 40 °F,"Thursday Night: Mostly clear, with a low aroun..."
3,Friday,Sunny,High: 60 °F,"Friday: Sunny, with a high near 60. Calm wind ..."
4,FridayNight,Mostly Clear,Low: 42 °F,"Friday Night: Mostly clear, with a low around ..."
5,Saturday,Mostly Sunny,High: 64 °F,"Saturday: Mostly sunny, with a high near 64."
6,SaturdayNight,Slight ChanceRain,Low: 47 °F,Saturday Night: A 20 percent chance of rain af...
7,Sunday,Chance Rain,High: 63 °F,"Sunday: A chance of rain. Partly sunny, with ..."
8,SundayNight,Slight ChanceRain,Low: 48 °F,Sunday Night: A slight chance of rain. Partly...


In [15]:
#desc has the text that contains the value of period also
#trying to split the desc string such that the period part is removed
new = weather["desc"].str.split(": ",expand=True) 
print(new)

                0                                                  1
0       Overnight  Mostly clear, with a low around 40. Southeast ...
1        Thursday  Sunny, with a high near 56. Light and variable...
2  Thursday Night  Mostly clear, with a low around 40. West wind ...
3          Friday  Sunny, with a high near 60. Calm wind becoming...
4    Friday Night  Mostly clear, with a low around 42. West north...
5        Saturday                 Mostly sunny, with a high near 64.
6  Saturday Night  A 20 percent chance of rain after 10pm.  Mostl...
7          Sunday  A chance of rain.  Partly sunny, with a high n...
8    Sunday Night  A slight chance of rain.  Partly cloudy, with ...


In [16]:
#fetching the other part and replacing the desc variable 
weather["desc"]=new[1]

In [11]:
#updated dataframe
weather

,period,short_desc,temp,desc
0,Today,Sunny,High: 56 °F,"Sunny, with a high near 56. North wind 9 to 17..."
1,Tonight,Clear,Low: 40 °F,"Clear, with a low around 40. North northwest w..."
2,Thursday,Sunny,High: 57 °F,"Sunny, with a high near 57. Light and variable..."
3,ThursdayNight,Mostly Clear,Low: 40 °F,"Mostly clear, with a low around 40. West north..."
4,Friday,Sunny,High: 60 °F,"Sunny, with a high near 60. Calm wind becoming..."
5,FridayNight,Mostly Clear,Low: 42 °F,"Mostly clear, with a low around 42."
6,Saturday,Mostly Sunny,High: 63 °F,"Mostly sunny, with a high near 63."
7,SaturdayNight,Slight ChanceRain,Low: 48 °F,A slight chance of rain after 10pm. Mostly cl...
8,Sunday,Chance Rain,High: 63 °F,"A chance of rain. Mostly cloudy, with a high ..."


In [12]:
weather.to_excel("weather_details.xlsx",header=True)

In [ ]:
#https://www.dataquest.io/blog/web-scraping-tutorial-python/